![](title.png)


* Documentation: http://inferpy.readthedocs.io

* Source code: https://github.com/PGM-Lab/InferPy

* Package: https://pypi.org/project/inferpy/


# Overview

InferPy is a probabilistic programming language that:
* allows easy and fast prototyping of probabilistic models
* is built on top of Edward and TensorFlow
* runs seamlessly on CPUs and GPUs


![](architecture.png)


Tensorflow is a framework that performs operations with tensors (https://www.tensorflow.org/) in an efficient way:

In [ ]:
import tensorflow as tf

A = tf.constant([[2.,3.,4.],[2.,3.,4.]])
B = tf.constant([[1.,2.,0.4],[10.,3.,4.]])

AB = tf.matmul(A,B, transpose_b=True)

AB


Note: for running each code-cell, select it and click shift+enter

In [ ]:
sess = tf.Session()
sess.run(AB)

Edward allows the definition of probabilistic models (http://edwardlib.org/). Inferpy aims to be its simplification 

In [ ]:
import edward as ed

a = ed.models.Poisson(5.) 
b = ed.models.Beta(2.,2.)

x = ed.models.Normal(loc=tf.matmul(tf.ones([3,2]), (tf.eye(2)*[2+a, 100*b])), scale=1.)


x.sample()

# Inferpy models

For using InferPy, the package should be imported as follows

In [ ]:
import inferpy as inf 

inf is the alias of the package, which will be used for simplicity. The inferpy version can be displayed with the following command:

In [ ]:
print(inf.VERSION)

Generate some toy data:

In [ ]:
N=1000 # number of observations
x_train = inf.models.Normal(10, scale=1).sample(N)

In [ ]:
print(x_train)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(x_train)
plt.title("Gaussian Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

fig = plt.gcf()

Definition of the model for learning the parameters (i.e. the mean): 

![](simple_model.png)



In [ ]:
with inf.ProbModel() as m:
    # prior (latent variable)
    theta = inf.models.Normal(loc=0, scale=1)

    # observed variable
    with inf.replicate(size=N):
        x = inf.models.Normal(loc = theta, scale=1, observed=True)


Encapsulate the training data in a dictionary

In [ ]:
data = {x.name : x_train}

Make inference:

In [ ]:
#compile and make inference
m.compile()
m.fit(data)

In [ ]:
print(m.posterior(theta))

# Random variables

InferPy allows the definition of variabels following the distributions below:

In [ ]:
inf.models.ALLOWED_VARS

For example:

In [ ]:
a = inf.models.Poisson(5.) 
b = inf.models.Beta(2.,2.)
c = inf.models.Categorical(probs=[0.3,0.2,0.5])

Samples can be obtained:

In [ ]:
a.sample(10)

In [ ]:
b.sample(10)

In [ ]:
c.sample(10)

Distribution matrix can me defined:

![](shape.png)


In [ ]:
with inf.replicate(size=10):
    x = inf.models.Normal(loc=[0,0,0], scale=[1,1,1])
    
print(x)

In [ ]:
with inf.replicate(size=10):
    x = inf.models.Normal(loc=0, scale=1, dim=3)
print(x)

In [ ]:
x.sample()

The parameters can be other variables:

In [ ]:
with inf.replicate(size=3):
    x = inf.models.Normal(loc=[2+a, 100*b], scale=1)
x.sample()

In [ ]:
x.shape

# Probabilistic models

A probabilistic model in InferPy is basically a set of random variables:


In [ ]:
with inf.ProbModel() as m:
    theta = inf.models.Beta(0.5,0.5)
    z =  inf.models.Categorical(probs=[theta, 1-theta], name="z")


m.sample()

An alternative definition:

In [ ]:
theta = inf.models.Beta(0.5,0.5)
z =  inf.models.Categorical(probs=[theta, 1-theta], name="z")

m = inf.ProbModel(varlist=[theta,z])
m.sample()



We might define models with complex dependencies between their variables

In [ ]:
# Categorical variable depending on another categorical variable

with inf.ProbModel() as m2:
    y =  inf.models.Categorical(probs=[0.4,0.6], name="y")
    x = inf.models.Categorical(probs=inf.case({y.equal(0): [0.0, 1.0],
                                               y.equal(1): [1.0, 0.0] }), name="x")
m2.sample()


In [ ]:
# Categorical variable depending on a Normal distributed variable

with inf.ProbModel() as m3:
    a = inf.models.Normal(0,1, name="a")
    b = inf.models.Categorical(probs=inf.case({a>0: [0.0, 1.0],
                                               a<=0: [1.0, 0.0]}), name="b")
m3.sample()


In [ ]:
# Normal distributed variable depending on a Categorical variable

with inf.ProbModel() as m4:
    d =  inf.models.Categorical(probs=[0.4,0.6], name="d")
    c = inf.models.Normal(loc=inf.case({d.equal(0): 0.,
                                        d.equal(1): 100.}), scale=1., name="c")
m4.sample()

# Inference

Linear Factor Model (PCA)
![](pca.png)

The model is defined as follows:

In [ ]:
K, d, N = 5, 10, 200

# model definition
with inf.ProbModel() as m:
    #define the weights
    with inf.replicate(size=K):
        w = inf.models.Normal(0, 1, dim=d)

    # define the generative model
    with inf.replicate(size=N):
        z = inf.models.Normal(0, 1, dim=K)
        x = inf.models.Normal(inf.matmul(z,w), 1.0, observed=True, dim=d)

m.compile()


Some training data is generated:

In [ ]:
x_train = inf.models.Normal(10,1,dim=d).sample(N)
data = {x.name : x_train}

Inference can be done with the default parameters:

In [ ]:
m.compile()
m.fit(data)
m.posterior(z)

The Q model and the inference algorithm can be specified:

In [ ]:
qw = inf.Qmodel.Normal(w)
qz = inf.Qmodel.Normal(z)

qmodel = inf.Qmodel([qw, qz])

m.compile(infMethod="KLqp", Q=qmodel)
m.fit(data)
m.posterior(z)

In [ ]:
inf.INF_METHODS